In [1]:
import os
import sys
import pandas as pd


In [2]:
review_meta = pd.read_csv("../../data/review_metadata.csv")

/tmp/ipykernel_9294/145584005.py:1: DtypeWarning: Columns (9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  review_meta = pd.read_csv("../../data/review_metadata.csv")


In [3]:
review_meta.head()

,Unnamed: 0,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,vote,...,also_view,main_cat,similar_item,date,price,details,software_category,Licensing_Fee,Implemention_cost,Maintenance_cost
0,0,5.0,False,"07 23, 2008",A8IOST6U6WH9B,0615179088,C. Radey,Human Japanese is a truly superb introduction ...,Human Japanese,12,...,"['B00N5EXLMC', '0976998122', '4789014401', '06...",Software,NaN,</div>,39.94,NaN,Education & Reference,0.008,19.97,3.994
1,1,5.0,False,"06 4, 2008",A1MUV9F35OROS5,0615179088,D. Abel,I got Human Japanese as a demo from its websit...,Best Japanese Program Available,11,...,"['B00N5EXLMC', '0976998122', '4789014401', '06...",Software,NaN,</div>,39.94,NaN,Education & Reference,0.008,19.97,3.994
2,2,4.0,False,"04 8, 2008",A27PAMABWVQ892,0615179088,piepiepie75,My first experience with Human Japanese was th...,Better than the Human Japanese 1...but not muc...,99,...,"['B00N5EXLMC', '0976998122', '4789014401', '06...",Software,NaN,</div>,39.94,NaN,Education & Reference,0.008,19.97,3.994
3,3,5.0,False,"03 26, 2008",A3HWWVK0L3JEKF,0615179088,K. Grier,This is the first language software that I hav...,Great Product,4,...,"['B00N5EXLMC', '0976998122', '4789014401', '06...",Software,NaN,</div>,39.94,NaN,Education & Reference,0.008,19.97,3.994
4,4,5.0,False,"02 20, 2008",A3NO2V2JU4Y8UY,0615179088,H. Granat,Human japanese is the best pc program for lear...,Love it!,2,...,"['B00N5EXLMC', '0976998122', '4789014401', '06...",Software,NaN,</div>,39.94,NaN,Education & Reference,0.008,19.97,3.994


In [19]:
len(review_meta)

145863

In [23]:
review_meta.columns.tolist()

['Unnamed: 0',
 'overall',
 'verified',
 'reviewTime',
 'reviewerID',
 'asin',
 'reviewerName',
 'reviewText',
 'summary',
 'vote',
 'category',
 'tech1',
 'description',
 'fit',
 'title',
 'also_buy',
 'tech2',
 'brand',
 'feature',
 'rank',
 'also_view',
 'main_cat',
 'similar_item',
 'date',
 'price',
 'details',
 'software_category',
 'Licensing_Fee',
 'Implemention_cost',
 'Maintenance_cost']

In [25]:
review_meta.description[0]

"['<DIV>Human Japanese is software for Windows PCs that presents the Japanese language from square one in a warm, engaging tone. It packs more than 1,800 recordings of vocabulary words and example sentences, an instantly searchable dictionary for vocabulary words and grammar terms, and dozens of interactive reviews. While other products focus on memorizing vocabulary, Human Japanese draws the student into the real mechanics of the language, while maintaining a warm tone and a sense of humor. The goal of Human Japanese is not just to turn out students capable of repeating canned phrases, but to lay the foundation for real linguistic and cultural understanding.</DIV>', '<B>Brian Rak</B> has been writing about the Japanese language for more than 10 years. Since the first Human Japanese product was released in 1999, he has maintained a web site where readers can access a forum to interact with the author and other students.']"

### Basic Analysis
<br>Idea: We use cluster data on sentiment and total price, using kmeans clustering.</br>
<br>Sentiments are generated from review data.</br>

In [8]:
import nltk

from nltk.sentiment.vader import SentimentIntensityAnalyzer

from nltk.corpus import stopwords

from nltk.tokenize import word_tokenize

from nltk.stem import WordNetLemmatizer

nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /home/theo/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /home/theo/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /home/theo/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /home/theo/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /home/theo/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /home/theo/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /home/the

True

In [5]:
cols = ['verified','reviewText', 'overall', 'price', 'rank', 'Licensing_Fee', 'Implemention_cost', 'Maintenance_cost']
data = review_meta[cols]


In [6]:
def preprocess_text(text):
    # Tokenize the text
    text = str(text)
    tokens = word_tokenize(text.lower())
    # Remove stop words
    filtered_tokens = [token for token in tokens if token not in stopwords.words('english')]
    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    # Join the tokens back into a string
    processed_text = ' '.join(lemmatized_tokens)
    return processed_text
data['reviewText'] = data['reviewText'].apply(preprocess_text)
data.head()

/tmp/ipykernel_9294/699602883.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['reviewText'] = data['reviewText'].apply(preprocess_text)


,verified,reviewText,overall,price,rank,Licensing_Fee,Implemention_cost,Maintenance_cost
0,False,human japanese truly superb introduction outsi...,5.0,39.94,"3,689 in Software (",0.008,19.97,3.994
1,False,got human japanese demo website . within week ...,5.0,39.94,"3,689 in Software (",0.008,19.97,3.994
2,False,first experience human japanese first version ...,4.0,39.94,"3,689 in Software (",0.008,19.97,3.994
3,False,first language software purchased love ! way i...,5.0,39.94,"3,689 in Software (",0.008,19.97,3.994
4,False,human japanese best pc program learning japane...,5.0,39.94,"3,689 in Software (",0.008,19.97,3.994


In [9]:
analyzer = SentimentIntensityAnalyzer()

# create get_sentiment function
def get_sentiment(text):
    scores = analyzer.polarity_scores(text)
    sentiment = 1 if scores['pos'] > 0 else 0
    return sentiment

data['sentiment'] = data['reviewText'].apply(get_sentiment)

/tmp/ipykernel_9294/2606887816.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['sentiment'] = data['reviewText'].apply(get_sentiment)


In [10]:
data.head()

,verified,reviewText,overall,price,rank,Licensing_Fee,Implemention_cost,Maintenance_cost,sentiment
0,False,human japanese truly superb introduction outsi...,5.0,39.94,"3,689 in Software (",0.008,19.97,3.994,1
1,False,got human japanese demo website . within week ...,5.0,39.94,"3,689 in Software (",0.008,19.97,3.994,1
2,False,first experience human japanese first version ...,4.0,39.94,"3,689 in Software (",0.008,19.97,3.994,1
3,False,first language software purchased love ! way i...,5.0,39.94,"3,689 in Software (",0.008,19.97,3.994,1
4,False,human japanese best pc program learning japane...,5.0,39.94,"3,689 in Software (",0.008,19.97,3.994,1


### Clustering on Price and Sentiment

In [15]:
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler, RobustScaler

In [16]:
scaler = StandardScaler()
features = data[['price','sentiment']]
scaled_features = scaler.fit_transform(features)

In [17]:
kmeans = KMeans(n_clusters=2)
dbscan = DBSCAN(eps=0.3)

In [18]:
kmeans.fit(scaled_features)
dbscan.fit(scaled_features)

/home/theo/anaconda3/envs/Societal_Computing/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
